In [3]:
import math
import os
import numpy as np
import librosa as lb
import soundfile as sf
import pyaudio

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

from VAE_1 import VAE_1

In [2]:
def play_wave(wave, sr):
    # initialize PyAudio
    p = pyaudio.PyAudio()

    # open a stream
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=sr,
                    output=True)

    # play audio
    stream.write(wave.tobytes())

    # stop stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

In [4]:
model = VAE_1(32)
model.load_state_dict(torch.load('model_1.pth'))

In [ ]:
def inference(model, mu, logvar):
    model.eval()
    z = model.reparameterize(mu, torch.tensor(logvar*2))
    with torch.no_grad():
        return model.decoder(z)
    
reconstructed_x = inference(model, torch.zeros((32)), torch.ones((32)))

reconstructed_x_np = reconstructed_x[0,0,:,:].numpy()

lb.display.specshow(reconstructed_x_np, sr = 44100)

inverted = lb.feature.inverse.mfcc_to_audio(reconstructed_x_np, sr=44100, n_mels=256, n_fft=512, hop_length=256, lifter=0, dct_type=3)
play_wave(inverted, 44100)


In [ ]:
# def load_random_sample(sample_type, paths_to_samples = ):
    

In [ ]:
VIEW_SHAPE = 1, 1, 256, 100
INVERSE_MFCC_PARAMS = {
    'sr': 44100,
    'n_mels': 256,
    'n_fft': 512,
    'hop_length': 256,
    'lifter': 0,
    'dct_type': 3
}

In [ ]:
kick = torch.from_numpy(mfccs[0]).view(1, 1, 256,100)


print(kick.shape)

reconstructed_x, mu, logvar = model.forward(kick)

reconstructed_x_np = reconstructed_x[0,0,:,:].detach().numpy()[:,:-2]


lb.display.specshow(reconstructed_x_np, sr = 44100)



inverted_orig = lb.feature.inverse.mfcc_to_audio(mfccs[0], sr=44100, n_mels=256, n_fft=512, hop_length=256, lifter=0, dct_type=3)
inverted = lb.feature.inverse.mfcc_to_audio(reconstructed_x_np, sr=44100, n_mels=256, n_fft=512, hop_length=256, lifter=0, dct_type=3)

play_wave(inverted_orig, 44100)
play_wave(inverted, 44100)

reconstructed_x = inference(model, mu + 1, logvar + 0.5)
reconstructed_x_np = reconstructed_x[0,0,:,:].detach().numpy()[:,:-2]

inverted = lb.feature.inverse.mfcc_to_audio(reconstructed_x_np, sr=44100, n_mels=256, n_fft=512, hop_length=256, lifter=0, dct_type=3)
lb.display.specshow(reconstructed_x_np, sr = 44100)

play_wave(inverted, 44100)

print(np.mean(mu.detach().numpy()))

In [130]:


def mfcc_to_wave(mfcc):
    wave = lb.feature.inverse.mfcc_to_audio(mfcc, **INVERSE_MFCC_PARAMS)
    return wave

kick_1_np = mfccs[20]
kick_1_torch = torch.from_numpy(kick_1_np).view(*VIEW_SHAPE)
kick_1_reconstructed, kick_1_mu, kick_1_logvar = model.forward(kick_1_torch)

kick_2_np = mfccs[-13]
kick_2_torch = torch.from_numpy(kick_2_np).view(*VIEW_SHAPE)
kick_2_reconstructed, kick_2_mu, kick_2_logvar = model.forward(kick_2_torch)

interpolated_mu = (kick_1_mu + kick_2_mu) / 2
interpolated_logvar = (kick_1_logvar + kick_2_logvar) / 2

reconstructed_interpolation = inference(model, interpolated_mu, interpolated_logvar)
# The :, :-2 indexing is because of the reconstruction after padding.
reconstructed_interpolation_np = reconstructed_interpolation[0, 0, :, :].detach().numpy()[:, :-2]
reconstructed_interpolation_wave = mfcc_to_wave(reconstructed_interpolation_np)


play_wave(mfcc_to_wave(kick_1_np), INVERSE_MFCC_PARAMS['sr'])
play_wave(mfcc_to_wave(kick_2_np), INVERSE_MFCC_PARAMS['sr']) 
play_wave(reconstructed_interpolation_wave, INVERSE_MFCC_PARAMS['sr'])

C:\Users\llama\AppData\Local\Temp\ipykernel_12592\4034893152.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z = model.reparameterize(mu, torch.tensor(logvar*2))
